In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from scipy import stats
from sklearn.metrics import mean_squared_error


from jupyterthemes import jtplot
jtplot.style()

from wrangle import wrangle_zillow, split_zillow, scale_data

import explore

plt.rcParams['figure.figsize'] = (4, 2)

# Analyzing and predicting customer churn
## Key findings:
-
- 

# Data acquisition and prep
- Data acquired from zillow sql database

In [2]:
zillow = wrangle_zillow()
zillow.head()

,square_feet,fips,latitude,longitude,lotsizesquarefeet,regionidzip,tax_value,taxes,transactiondate,age_in_years,Bathrooms,Bedrooms,tax_rate
parcelid,,,,,,,,,,,,,
11721753,1316.0,6037.0,34033180.0,-118355396.0,5672.0,95997.0,205123.0,2627.48,2017-07-21,98.0,2,2,1.28
11289917,1458.0,6037.0,34686163.0,-118113100.0,8284.0,97318.0,136104.0,2319.90,2017-06-23,51.0,2,2,1.70
11637029,1766.0,6037.0,34073222.0,-118390965.0,25082.0,96086.0,810694.0,9407.01,2017-07-28,41.0,2,2,1.16
11705026,1421.0,6037.0,33999877.0,-118291863.0,6707.0,96018.0,35606.0,543.69,2017-06-30,110.0,1,1,1.53
14269464,2541.0,6059.0,33694636.0,-117912245.0,4975.0,96958.0,880456.0,9819.72,2017-06-01,18.0,3,3,1.12


In [3]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32737 entries, 11721753 to 10713993
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   square_feet        32737 non-null  float64
 1   fips               32737 non-null  float64
 2   latitude           32737 non-null  float64
 3   longitude          32737 non-null  float64
 4   lotsizesquarefeet  32737 non-null  float64
 5   regionidzip        32737 non-null  float64
 6   tax_value          32737 non-null  float64
 7   taxes              32737 non-null  float64
 8   transactiondate    32737 non-null  object 
 9   age_in_years       32737 non-null  float64
 10  Bathrooms          32737 non-null  object 
 11  Bedrooms           32737 non-null  object 
 12  tax_rate           32737 non-null  float64
dtypes: float64(10), object(3)
memory usage: 3.5+ MB


## Exploration

In [ ]:
# splitting columns between categorical and quantitative for explore functions
cat_vars = ['fips', 'Bedrooms', 'Bathrooms']

quant_vars = ['square_feet',                         
              'lotsizesquarefeet',              
              'tax_value',
              'taxes',              
              'age_in_years',
              ]

In [ ]:
#explore.py function to visualize single category data
explore.explore_univariate(zillow, cat_vars, quant_vars)

##### Univariate Takeaways
- 

In [ ]:
train, validate, test = split_zillow(zillow)

In [ ]:
#explore function to compare variables to the target of 'churn'
explore.explore_bivariate(train,'tax_value', cat_vars, quant_vars)

##### Bivariate Takeaways
- 

In [ ]:
#Narrowing down variables for use with multivariate function. This function compares two categories and separates churn by hue
cat_vars = ['Month_to_month','Fiber_optic','DSL','No_internet']
quant_vars = ['tenure','monthly_charges']
explore.explore_multivariate(train,'churn', cat_vars,quant_vars)

##### Multivariate Takeaways
- Fiber optic customers have much higher minimum monthly charges than other customers
- Half of churned month-to-month customers paid over $70 a month for service

### 1. Are the monthly charges between churned customers significantly higher from unchurned customers?

In [ ]:
null_hypothesis = "The means of monthly_charges are the same for churn vs. no churn"
alternative_hypothesis = "Churned customers had higher monthly_charges than unchurned"
confidence_level = .95
a = 1 - confidence_level 

In [ ]:
#subsetting churned observations from train data
churned = train[train.churn == 1]
no_churn = train[train.churn == 0]
churned_fiber_optic = churned[churned.Fiber_optic == 1]

In [ ]:
# % of total churned: month-to-month customers 87% , fiber optic customers 69%, % of churned month-to-month fiber optic customers 89%
(churned.Month_to_month == 1).mean(), (churned.Fiber_optic == 1).mean(), (churned_fiber_optic.Month_to_month == 1).mean()

In [ ]:
# % of total churned, month-to-month fiber optic customers. 61% ot total churn
((churned.Month_to_month == 1) & (churned.Fiber_optic == 1)).mean()

In [ ]:
#one tailed two sample t-test for independence
t, p = stats.ttest_ind(churned.monthly_charges, no_churn.monthly_charges)
t, p

In [ ]:
if (p/2) < a and t > 0:
    print("Reject the null hypothesis")
else:
    print("We fail to reject the null hypothesis")

In [ ]:
print('The mean monthly charges for churned customers is significantly greater than unchurned')

In [ ]:
#verification: average monthly charges for churned customers is approximately 20% higher than unchurned on the train data set
churned.monthly_charges.mean(), no_churn.monthly_charges.mean()

### 2. Is Churn independent of the Fiber Optic service?

In [ ]:
null_hypothesis = "Churn is independent of the Fiber Optic service"
alternative_hypothesis = "Churn depends on Fiber Optic service subscription"
confidence_level = .95
a = 1 - confidence_level

In [ ]:
observed = pd.crosstab(train.churn, train.Fiber_optic)
observed

In [ ]:
#chi2 test between two categorical variables
chi2, p, degf, expected = stats.chi2_contingency(observed)
chi2, p, degf, expected

In [ ]:
if p < a:
    print('We reject the null')
else:
    print("we fail to reject the null")

In [ ]:
print('There is enough evidence to suggest that churn is dependent on whether the customer has Fiber Optic service or not')

### 3. Is there a correlation between monthly charges and tenure for churned fiber optic customers?

In [ ]:
null_hypothesis = "There is no correlation between monthly charges and tenure for churned fiber optic customers"
alternative_hypothesis = "There is a positive correlation between monthly charges and tenure for churned fiber optic customers"
confidence_level = .95
a = 1 - confidence_level

#Pearson correlation test between two continuous variables
r, p = stats.pearsonr(churned_fiber_optic.tenure, churned_fiber_optic.monthly_charges)
r, p

In [ ]:
if p < a:
    print('We reject the null')
else:
    print("we fail to reject the null")
    
print('Pearson correlation = ', r)
print('There is a weak-moderate positive correlation between tenure and monthly charges for churned fiber optic customers')

In [ ]:
#correlation visualization
jtplot.style()

sns.relplot(x='tenure',y='monthly_charges',data=churned_fiber_optic)
plt.title('Tenure vs Monthly Charges for churned fiber optic customers')
plt.text(12,115, f'r = {r:.3f}')

# Modeling and Evaluation
- Creating all 4 types of classification models using mostly default settings then choosing the best performing
- get_metrics function from prepare.py

In [ ]:
#setting baseline
train.churn.value_counts()

In [ ]:
#baseline is 0 == no churn
train['baseline_prediction'] = 0
train.head(1)

In [ ]:
baseline_accuracy = (train.churn == train.baseline_prediction).mean()
baseline_accuracy

### Baseline accuracy ~ 73%

In [ ]:
#Narrowing down features based on previous hypothesis testing. Partner and dependents were included just based on analysis from previous project
features = ['tenure','monthly_charges','partner','dependents','Month_to_month',
            'One_year_contract','Two_year_contract','DSL','Fiber_optic','No_internet']

X_train = train[features]
y_train = train.churn 

X_validate = validate[features]
y_validate = validate.churn

X_test = test[features]
y_test = test.churn

## Model 1: Decision Tree

In [ ]:
clf = DecisionTreeClassifier(max_depth=5, random_state=123)
clf = clf.fit(X_train,y_train)

In [ ]:
#training data report
decision_tree_report = get_metrics(clf, X_train, y_train)
decision_tree_report

In [ ]:
#validate data report
decision_tree_report_validate = get_metrics(clf,X_validate,y_validate)
decision_tree_report_validate

## Model 2: Random Forest

In [ ]:
rf = RandomForestClassifier(bootstrap=True,
                            class_weight=None,
                            criterion='gini',
                            min_samples_leaf=5,
                            n_estimators=100,
                            max_depth=5,
                            random_state=123)
rf = rf.fit(X_train, y_train)

In [ ]:
#training data report
rf_report = get_metrics(rf,X_train,y_train)
rf_report

In [ ]:
#validate data report
rf_report_validate = get_metrics(rf,X_validate,y_validate)
rf_report_validate

## Model 3: KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train, y_train)

#training data report
knn_report = get_metrics(knn,X_train,y_train)
knn_report

In [ ]:
#validate data report
knn_report_validate = get_metrics(knn,X_validate,y_validate)
knn_report_validate

## Model 4: Logistic Regression

In [ ]:
logit = LogisticRegression(C=1, class_weight='balanced',
                           random_state=123,solver='liblinear')
logit = logit.fit(X_train, y_train)

#training data report
log_report = get_metrics(logit,X_train,y_train)
log_report

In [ ]:
#validate data report
log_report_validate = get_metrics(logit,X_validate,y_validate)
log_report_validate

# Logistic Regression model chosen for it's high recall compared to other models
- False negative churn predictions are more costly so a high recall is necessary

In [ ]:
#test data report
log_report_test = get_metrics(logit, X_test,y_test)
log_report_test

In [ ]:
#creating churn predictions and probabilities using test dataset and log model and outputing to .csv
telco_predict = telco[features]
prediction = logit.predict(telco_predict)
prediction_probability = logit.predict_proba(telco_predict)
prediction, prediction_probability

In [ ]:
#new predictions dataframe
predicted_df = pd.concat([telco.customer_id,pd.DataFrame(prediction_probability[0:,1]),pd.DataFrame(prediction)],axis=1)
predicted_df.columns = ['customer_id', 'probability_of_churn', 'churn_prediction']
predicted_df

In [ ]:
predicted_df.to_csv('churn_predictions.csv')

In [ ]:
#accuracy of predictions compared to actual churn
(telco.churn == predicted_df.churn_prediction).mean()

# Key Takeaways
- We need to address the high rate of churn for our month-to-month and fiber optic customers
- Current models aren't much more accurate than baseline but logistic regression model had highest recall of 0.82
- With more time I would tweak each model to maximize performance and possibly add features